# Singular Value Decomposition of Coupled Fields.

# Import necessary libraries

In [ ]:
import numpy as np
import scipy as sc
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
from matplotlib.ticker import MultipleLocator

In [ ]:
# Open the netCDF file and extract the sst variable

dm = nc.Dataset('input your file 1') # file 1
ds = nc.Dataset('input your file 2') # file 2

# Define your vaiables

var_1 = dm.variables['Your Variable 1'][:] 
var_2 = ds.variables['Your Variable 2'][:]

# Extract Longitude, Latitude and time values

lon = ds.variables['Your Longitude variable Name'][:]
lat = ds.variables['Your Latitude variable Name'][:]
time = ds.variables['Your Time variable Name']


# Reshape the Datafiles to 2-D arrays

In [ ]:
# Reshape the sst array into a 2D array

var_1_2d = sst.reshape(sst.shape[0], -1)
var_2_2d = mfc.reshape(mfc.shape[0], -1)


In [ ]:
# Create a list of times and locations for sst

times = np.arange(sst.shape[0])
lons, lats = np.meshgrid(ds.variables['Your Longitude variable Name'][:], ds.variables['YYour Latitude variable Name'][:])
locations = [(lon, lat) for lon, lat in zip(lons.flatten(), lats.flatten())]

# Create pandas dataframe to easilly manipulate the datasets

In [ ]:
# Create a pandas DataFrame with the reshaped SST data

dfs = pd.DataFrame(data=sst_2d, index=times, columns=locations)


In [ ]:
# Create a pandas DataFrame with the reshaped MFC data

dfm = pd.DataFrame(data=mfc_2d, index=times, columns=locations)


# Reshape the dataset without NAN values

In [ ]:
# Drop columns containing no values for sst

dfs.dropna(axis=1, how='all', inplace=True)
print('Dimension of modified sst array ; ',dfs.shape)


# Convert the dataframe to numpy array

In [ ]:
# Convert the DataFrame to a NumPy array

ds1_array = dfs.to_numpy()
dm1_array = dfm.to_numpy()

In [ ]:
# shape of new numpy array

print(ds1_array.shape)
print(dm1_array.shape)

# Detrend the Datasets

In [ ]:
# detrend the data

ds1_array = sc.signal.detrend(ds1_array.T, type='constant')
ds1_array = ds1_array.T
print('Dimension of modified sst array :', ds1_array.shape)

dm1_array = sc.signal.detrend(dm1_array.T, type='constant')
dm1_array = dm1_array.T
print('Dimension of modified mfc array :', dm1_array.shape)

# Create Covariance Matrix

In [ ]:
# covariance matrix

c = ds1_array.T.dot(dm1_array) # sst x mfc
print('Dimension Of Covariance Matrix :', c.shape)

# Perform Singular Value Decomposition

In [ ]:
# perform svd

u, s, vt = np.linalg.svd(c,full_matrices=True)
print('shape of u', u.shape)
print('shape of s', s.shape)
print('shape of vt', vt.shape)

# Calculate the square covariance fraction

In [ ]:
# Calculate SCF for each mode

SCF = (s**2) / np.sum(s**2)

# Find the Expansion Coefficients

In [ ]:
# Expansion Coefficients

a = ds1_array.dot(u) # sst
b = dm1_array.dot(vt) # mfc

T = np.linspace(1980,2020,40) # years

# Calculate the Correlation Coefficients

In [ ]:
# Extract the first expansion coefficients of SST and MFC

a_first_three = a[:, 0]
b_first_three = b[:, 0]

# Calculate the correlation matrix

corr_coef = np.corrcoef((a_first_three.T), (b_first_three.T))
mode_index = 1

print(f"Mode {mode_index}: SCF={SCF[0]*100:.2f}%, corr_coef={corr_coef[0,1]:.2f}")

#detrended

corr_coef = np.corrcoef(sc.signal.detrend(a_first_three.T), sc.signal.detrend(b_first_three.T))
print(f"Mode {mode_index} detrended: SCF={SCF[0]*100:.2f}%, corr_coef={corr_coef[0,1]:.2f}")

In [ ]:
# Extract the second expansion coefficients of SST and MFC

a_first_three = a[:, 1:2]
b_first_three = b[:, 1:2]

# Calculate the correlation matrix

corr_coef = np.corrcoef((a_first_three.T), (b_first_three.T))
mode_index = 2
print(f"Mode {mode_index}: SCF={SCF[1]*100:.2f}%, corr_coef={corr_coef[0,1]:.2f}")

#detrended

corr_coef = np.corrcoef(sc.signal.detrend(a_first_three.T), sc.signal.detrend(b_first_three.T))
print(f"Mode {mode_index} detrended: SCF={SCF[1]*100:.2f}%, corr_coef={corr_coef[0,1]:.2f}")

In [ ]:
# Extract the third expansion coefficients of SST and MFC

a_first_three = a[:, 2:3]
b_first_three = b[:, 2:3]

# Calculate the correlation matrix

corr_coef = np.corrcoef((a_first_three.T), (b_first_three.T))
mode_index = 3
print(f"Mode {mode_index}: SCF={SCF[2]*100:.2f}%, corr_coef={corr_coef[0,1]:.2f}")

#detrended

corr_coef = np.corrcoef(sc.signal.detrend(a_first_three.T), sc.signal.detrend(b_first_three.T))
print(f"Mode {mode_index} detrended: SCF={SCF[2]*100:.2f}%, corr_coef={corr_coef[0,1]:.2f}")

# Plot the Timeseries of your variables

In [ ]:
# plot

fig = plt.figure(figsize=(8, 6))

# Plot the expansion coefficients for the first three modes
for i in range(3):
    plt.plot(T, a[:, i], label=f'Mode {i+1} Var name')
    plt.plot(T, b[:, i], label=f'Mode {i+1} Var name')
    plt.axhline(y=0, color='black', linestyle='--')

    # Add labels and legend
    plt.xlabel('X label')
    plt.ylabel('Y label')
    plt.title('Title')
    plt.legend()

    # Save and show the figure
    plt.tight_layout()
    plt.show()


# Reshape your datasets back to the original dimension

In [ ]:
u = u.T

In [ ]:
# Create a pandas DataFrame with the reshaped sst data

dfs = pd.DataFrame(data=sst_2d, index=times, columns=locations)
dfs.shape

In [ ]:
# Get the list of columns with NaN values
nan_columns = dfs.columns[dfs.isna().any()].tolist()
print(len(nan_columns))

In [ ]:
# Get the index values of the columns that contain NaN values
nan_column_indices = dfs.columns.get_indexer(nan_columns)
nan_column_indices

In [ ]:
c=np.nan

In [ ]:
u = u.astype(float)

In [ ]:
# reshaping the modified array to the original dimension of the array

for x in nan_column_indices:
    u = np.insert(u, x, c, axis=1)
u.shape

# Plot the spatial patterns

In [ ]:
levels = np.linspace(-0.03, 0.06, 10)

for i in range(3):
    sst_mode = u[i].reshape(sst.shape[1], sst.shape[2])  # shape: (lat, lon)

    # Plot the singular modes as contour maps with land
    fig = plt.figure(figsize=(10, 8))

    # Plot sst_mode
    ax2 = fig.add_subplot(111, projection=ccrs.PlateCarree())
    ax2.set_extent([dm.variables["Your Longitude variable Name"][:].min(), dm.variables["Your Longitude variable Name"][:].max(),
                    dm.variables["Your Latitude variable Name"][:].min(), dm.variables["Your Latitude variable Name"][:].max()], crs=ccrs.PlateCarree())
    ax2.coastlines()
    
    cf = ax2.contourf(dm.variables["Your Longitude variable Name"][:], dm.variables["Your Latitude variable Name"][:], sst_mode, levels=levels,
                      transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')

   

    ax2.set_title(f'Singular Mode {i + 1} Title',fontsize=10)
    
    # Add colorbar with label
    cbar = plt.colorbar(cf, shrink=0.37, pad=0.04)
    cbar.ax.tick_params(labelsize=8)
    

    # Add longitude and latitude gridlines
    gl = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                       linewidth=1, color='gray', alpha=0.005, linestyle='-')

    
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.xlabel_style = {'size': 9, 'color': 'black'}
    gl.ylabel_style = {'size': 9, 'color': 'black'}

    ax2.set_aspect(2.8)
    plt.tight_layout()
    plt.show()


In [ ]:
levels = np.linspace(-0.03, 0.06, 10)

for i in range(3):
    mfc_mode = vt[i].reshape(sst.shape[1], sst.shape[2])  # shape: (lat, lon)

    # Plot the singular modes as contour maps with land
    fig = plt.figure(figsize=(10, 8))

    # Plot sst_mode
    ax2 = fig.add_subplot(111, projection=ccrs.PlateCarree())
    ax2.set_extent([dm.variables["Your Longitude variable Name"][:].min(), dm.variables["Your Longitude variable Name"][:].max(),
                    dm.variables["Your Latitude variable Name"][:].min(), dm.variables["Your Latitude variable Name"][:].max()], crs=ccrs.PlateCarree())
    ax2.coastlines()
    cf = ax2.contourf(dm.variables["Your Longitude variable Name"][:], dm.variables["Your Latitude variable Name"][:], mfc_mode, levels=levels,
                      transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')

   

    ax2.set_title(f'Singular Mode {i + 1} Title',fontsize=10)
    
    # Add colorbar with label
    cbar = plt.colorbar(cf, shrink=0.37, pad=0.04)
    cbar.ax.tick_params(labelsize=8)
    

    # Add longitude and latitude gridlines
    gl = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                       linewidth=1, color='gray', alpha=0.005, linestyle='-')

    
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.xlabel_style = {'size': 9, 'color': 'black'}
    gl.ylabel_style = {'size': 9, 'color': 'black'}

    ax2.set_aspect(2.8)
    plt.tight_layout()
    plt.show()
